In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("sales_data.csv",index_col=0)

In [3]:
df["brand"] = df["brand"].apply(lambda x: x[:-1])

In [4]:
df.head()

,sku,product_line,brand,sales,price
SKU_ID,,,,,
1,Samsung Stereo Headset Wird EO-EG920BWEG,Phones Mobile Accessories,SAMSUNG,1,1359.0
2,HITACHI REF SBS RM-700AGPND4X-(DIA),Refrigerators,HITACHI,1,184408.0
3,SIEMENS DISHWASHER SN26L801IN,Dishwashers,SIEMENS,1,34425.0
4,Super General S/AC 1.5T SGSI185-3BE 3S,Air Conditioners,Super General,1,29750.0
5,Apple 12W USB Power Adapter,Tablets & Detachables,APPLE,1,1700.0


In [5]:
df.info(), df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2015 entries, 1 to 2015
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sku           2015 non-null   object 
 1   product_line  2015 non-null   object 
 2   brand         2015 non-null   object 
 3   sales         2015 non-null   int64  
 4   price         2015 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 94.5+ KB


(None, (2015, 5))

In [6]:
df = df.astype({"sku":"string","product_line":"string","brand":"string"})

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2015 entries, 1 to 2015
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sku           2015 non-null   string 
 1   product_line  2015 non-null   string 
 2   brand         2015 non-null   string 
 3   sales         2015 non-null   int64  
 4   price         2015 non-null   float64
dtypes: float64(1), int64(1), string(3)
memory usage: 94.5 KB


In [8]:
df["product_line"].unique()

<StringArray>
[    'Phones Mobile Accessories',                 'Refrigerators',
                   'Dishwashers',              'Air Conditioners',
         'Tablets & Detachables',               'Gaming Software',
              'Mobile Computing',            'Kitchen Appliances',
                   'MP3 Players',                 'Input Devices',
          'Earphones/Headphones',               'Home Appliances',
              'Washing Machines',               'Water Purifiers',
                    'Microwaves',            'Travel Accessories',
                  'Phones Fixed',       'Smart Phones (OS Based)',
                 'Computer Bags',                  'Shop in Shop',
        'Health & Personal Care',                  'Home Theatre',
                 'Phones Mobile',         'Lifestyle & Wearables',
               'Digital Cameras',                      'Security',
                        'TV LCD',                   'Peripherals',
                 'Storage Media',   'Printers & 

In [9]:
import re
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer


lemma = WordNetLemmatizer()
replacement = {
   re.compile(r"&|/"):" ",
   re.compile(r"( )+"):" ",
   re.compile(r" \(.*\)"):"",
   re.compile(r"^\s+|\s+$"):""
}

df["product_line_clean"] = df["product_line"].str.lower().replace(
    regex=replacement
    ).apply(
    lambda x: (" ".join(lemma.lemmatize(word) for word in x.split(" ")))
    )

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/slowgamer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
categories = df["product_line_clean"].unique()
categories

array(['phone mobile accessory', 'refrigerator', 'dishwasher',
       'air conditioner', 'tablet detachables', 'gaming software',
       'mobile computing', 'kitchen appliance', 'mp3 player',
       'input device', 'earphone headphone', 'home appliance',
       'washing machine', 'water purifier', 'microwave',
       'travel accessory', 'phone fixed', 'smart phone', 'computer bag',
       'shop in shop', 'health personal care', 'home theatre',
       'phone mobile', 'lifestyle wearable', 'digital camera', 'security',
       'tv lcd', 'peripheral', 'storage medium',
       'printer office solution', 'gaming hardware', 'imaging accessory',
       'speaker', 'pc accessory', 'phone mobile provider', 'cookware',
       'fan', 'entertainment accessory', 'desktop', 'dvd vcd content',
       'toy', 'networking', 'dummy laptop netbook tab',
       'cooling heating appliance', 'oven cooker', 'power',
       'air purifier', 'connected home housewares', 'audio system',
       'software', 'camcorde

In [11]:
df["brand_lower"] = df["brand"].str.lower()

brand_names = df["brand_lower"].unique()
brand_names

<StringArray>
[      'samsung',       'hitachi',       'siemens', 'super general',
         'apple',     'stuffcool',       'ubisoft',          'asus',
       'airplus',         'bajaj',
 ...
       'digisol',      'hindware',       'livpure',         'ricoh',
      'reliance',         'intel',        'ifrogz',          'drav',
        'amazon',       'mitashi']
Length: 151, dtype: string

In [12]:
df.to_parquet("sales_data.parquet")

In [13]:
from Levenshtein import ratio,distance

def text_to_list(txt,splitter=" ",lower=True, lemmatize=True):
    if lower:
        txt = txt.lower()
    
    if lemmatize:
        return [lemma.lemmatize(word) for word in txt.split(splitter)]
    
    return txt.split(splitter)

#max_win_score uses window size of category words and calculate Levenshtein similarity ratio (Windows shrink at the end)
#if score is >= 0.5 particuar brand df is return else all brands df
def max_win_score(cats,txt_ls):
    txt_n = len(txt_ls)
    cat_scores = {cat:0 for cat in cats}

    for cat in cats:
        cat_ls = cat.split(" ")
        n = len(cat_ls)

        for i in range(txt_n):
            temp = " ".join(txt_ls[i:i+n])

            cat_scores[cat] = max(cat_scores[cat],ratio(cat.lower(),temp.lower()))

    return cat_scores

def perm_avg_score(cat_ls,txt_ls):
    
    score = {word:0 for word in cat_ls}
        
    for word_cat in cat_ls:
        for word_txt in txt_ls:
            score[word_cat] = max(score[word_cat],ratio(word_cat,word_txt))
            
    return np.mean(list(score.values()))
#average_score take cartesian cross product, calculate Levenshtein similarity ratio
#and average max similarity ratio for each item
#if score is >= 0.5 particuar brand df is return else all brands df
def average_score(cats,txt_ls):
    txt_n = len(txt_ls)
    cat_scores = {cat:0 for cat in cats}
    
    for cat in cats:
        cat_ls = cat.split(" ")
        n = len(cat_ls)
        score = {word:0 for word in cat_ls}

        
        cat_scores[cat] = perm_avg_score(cat_ls,txt_ls)
        
    return cat_scores
            
def combine_scoring(cats,txt_ls):
    txt_n = len(txt_ls)
    cat_scores = {cat:0 for cat in cats}
    
    for cat in cats:
        cat_ls = cat.split(" ")
        n = len(cat_ls)
            
        for i in range(txt_n-n+1):
            temp = perm_avg_score(cat_ls,txt_ls[i:i+n])
            
            cat_scores[cat] = max(cat_scores[cat],temp)

    return cat_scores


#exact_match function first try exact matching of brand name in search text and return that brand or  dataframe
#if no exact match found, partial match is done using average_score or max_win_score

def exact_match(df,cat,txt,method):
   
    txt_ls = text_to_list(txt,lemmatize=False)
    ind = df[cat].isin(txt_ls)
    
    if ind.any():
        return df[ind]
    
    if method=="max_win_score":
        tp = max_win_score(df[cat].unique(),txt_ls)
    elif method=="average_score":
        tp = average_score(df[cat].unique(),txt_ls)
    else:
        tp = combine_scoring(df[cat].unique(),txt_ls)

    ele = max(tp.items(),key= lambda x:x[1])
    return df.loc[df[cat]==ele[0]].copy() if ele[1]>=0.5 else df.copy()

#partial match return top_scoring product_lines using average_score or max_win_score
def partial_match(df,cat,txt,method,lemmatize=True):
    
    top_ele=5
    txt_ls = text_to_list(txt)
    if method=="average_score":
        tp = average_score(df[cat].unique(),txt_ls)
    elif method=="average_score":
        tp = max_win_score(df[cat].unique(),txt_ls)
    else:
        tp = combine_scoring(df[cat].unique(),txt_ls)

    tp = sorted(tp.items(),key = lambda x: x[1],reverse=True)
    
    ind = df[cat].isin([x for x,y in tp if y>0.6] if tp[0][1]>0.6 else [x for x,y in tp])
    return df[ind].copy()



In [20]:
def inverse_partial_match(df,column,txt):
    threshold=0.9
    txt_ls = text_to_list(txt)
    txt_n = len(txt_ls)
    filter_vals = df[column].apply(lambda x: text_to_list(x))
    
    thresholds = {np.around(i,decimals=2):[] for i in np.arange(0,1.05,0.1)}
    
    for sku in filter_vals.index:
        txt_score = [0 for _ in range(txt_n)]
        for i in range(txt_n):
            for word in filter_vals[sku]:
                
                txt_score[i] = max(txt_score[i],ratio(txt_ls[i],word))
        txt_score.sort(reverse=True)   
    
        avg = np.mean(txt_score)
        thresholds[round(avg,1)].append(sku)

    res_id = []    
    for threshold in np.arange(1,(txt_n-1)/txt_n,-0.1):
        res_id.extend(thresholds[np.around(threshold,2)])
        if len(res_id)>=5:
            return df.loc[res_id]
        
    return df.loc[res_id] if len(res_id)>0 else df
    
    
    

In [24]:
text = "Samsung Galaxy J7 smartphone"
df_brand = exact_match(df,"brand_lower",text,"max_win_score")
df_pro = partial_match(df_brand,"product_line_clean",text,"average_score")
inverse_partial_match(df_pro,"sku",text)

,sku,product_line,brand,sales,price,product_line_clean,brand_lower
SKU_ID,,,,,,,
35,Samsung Galaxy J200G Black,Smart Phones (OS Based),SAMSUNG,15,7297.000000,smart phone,samsung
64,Samsung Galaxy J200G Gold,Smart Phones (OS Based),SAMSUNG,30,7335.033333,smart phone,samsung
111,Samsung Galaxy S7 Edge Blue,Smart Phones (OS Based),SAMSUNG,1,42900.000000,smart phone,samsung
165,SAMSUNG J7 Pro Black,Smart Phones (OS Based),SAMSUNG,98,20878.408160,smart phone,samsung
198,SAMSUNG J5 Prime 32GB Black,Smart Phones (OS Based),SAMSUNG,33,13479.787880,smart phone,samsung
203,SAMSUNG J 7 Max Black,Smart Phones (OS Based),SAMSUNG,100,17891.050000,smart phone,samsung
238,SAMSUNG J 7 Max Gold,Smart Phones (OS Based),SAMSUNG,91,17895.076920,smart phone,samsung
412,Samsung C9 Pro Black,Smart Phones (OS Based),SAMSUNG,17,31900.000000,smart phone,samsung
430,Samsung S8+ Gold,Smart Phones (OS Based),SAMSUNG,2,64900.000000,smart phone,samsung


In [22]:
text = "SAMSUNG washing machine TL 6KG WA60M4300HD/TL SLV" 
df_brand = exact_match(df,"brand_lower",text,"max_win_score")
df_pro = partial_match(df_brand,"product_line_clean",text,"average_score")
inverse_partial_match(df_pro,"sku",text)

,sku,product_line,brand,sales,price,product_line_clean,brand_lower
SKU_ID,,,,,,,
20,Samsung Wash/Dry 8/6K-WD80J6410,Washing Machines,SAMSUNG,6,54485.00000,washing machine,samsung
27,SAMSUNG W/M TL 6KG WA60M4300HD/TL SLV,Washing Machines,SAMSUNG,15,14990.00000,washing machine,samsung
179,SAMSUNG W/M TL 6.5KG WA65M4000HA/TL SLV,Washing Machines,SAMSUNG,4,20990.00000,washing machine,samsung
226,SAMSUNG W/M TL 7KG WA70M4500HL/TL BLU,Washing Machines,SAMSUNG,2,21990.00000,washing machine,samsung
230,SAMSUNG W/M FL 8KG WW80J4233KW/TL,Washing Machines,SAMSUNG,6,30982.00000,washing machine,samsung
407,SAMSUNG W/M TL 6.2KG WA62M4200HV/TL SLV,Washing Machines,SAMSUNG,6,18115.00000,washing machine,samsung
496,SAMSUNG W/M FL 6KG WW60M206LMA/TL WHT,Washing Machines,SAMSUNG,12,23962.00000,washing machine,samsung
847,SAMSUNG WM TL 9KG WA90J5730SS,Washing Machines,SAMSUNG,3,29742.00000,washing machine,samsung
880,SAMSUNG W/M TL 6.2KG WA62M4300HP/TL SLV,Washing Machines,SAMSUNG,7,16448.14286,washing machine,samsung


In [16]:
text = "SAMSUNG W/M TL 6KG WA60M4300HD/TL SLV" 
df_brand = exact_match(df,"brand_lower",text,"max_win_score")
df_pro = partial_match(df_brand,"product_line_clean",text,"average_score")
inverse_partial_match(df_pro,"sku",text)

,sku,product_line,brand,sales,price,product_line_clean,brand_lower
SKU_ID,,,,,,,
27,SAMSUNG W/M TL 6KG WA60M4300HD/TL SLV,Washing Machines,SAMSUNG,15,14990.00000,washing machine,samsung
179,SAMSUNG W/M TL 6.5KG WA65M4000HA/TL SLV,Washing Machines,SAMSUNG,4,20990.00000,washing machine,samsung
407,SAMSUNG W/M TL 6.2KG WA62M4200HV/TL SLV,Washing Machines,SAMSUNG,6,18115.00000,washing machine,samsung
880,SAMSUNG W/M TL 6.2KG WA62M4300HP/TL SLV,Washing Machines,SAMSUNG,7,16448.14286,washing machine,samsung
1864,SAMSUNG W/M TL 6.5KG WA65M4200HD/TL SLV,Washing Machines,SAMSUNG,1,19771.00000,washing machine,samsung


In [17]:
print(df.to_string())

                                             sku                   product_line          brand  sales          price         product_line_clean    brand_lower
SKU_ID                                                                                                                                                        
1       Samsung Stereo Headset Wird EO-EG920BWEG      Phones Mobile Accessories        SAMSUNG      1    1359.000000     phone mobile accessory        samsung
2            HITACHI REF SBS RM-700AGPND4X-(DIA)                  Refrigerators        HITACHI      1  184408.000000               refrigerator        hitachi
3                  SIEMENS DISHWASHER SN26L801IN                    Dishwashers        SIEMENS      1   34425.000000                 dishwasher        siemens
4         Super General S/AC 1.5T SGSI185-3BE 3S               Air Conditioners  Super General      1   29750.000000            air conditioner  super general
5                    Apple 12W USB Power Adapt

In [18]:
txt = 'Croma stereo Headphone'
brand_df = exact_match(df,"brand_lower",txt,"max_win_score")
product_df = partial_match(brand_df,"product_line_clean",txt,"")
inverse_partial_match(product_df,"sku",txt)

,sku,product_line,brand,sales,price,product_line_clean,brand_lower
SKU_ID,,,,,,,
13,Croma stereo Headphone CREA4209 IGH-2,Earphones/Headphones,CROMA,5,999.0,earphone headphone,croma
648,Croma stereo earphone CREA7288 SHCAMP,Earphones/Headphones,CROMA,17,499.0,earphone headphone,croma
1967,Croma Stereo Earphones EA7286 MD088L,Earphones/Headphones,CROMA,1,194.0,earphone headphone,croma
151,Croma sports earphone EA7287,Earphones/Headphones,CROMA,2,599.0,earphone headphone,croma
1515,Croma Leather Headphone EA4204 HP-0002,Earphones/Headphones,CROMA,2,494.0,earphone headphone,croma
